Анализируем данные с data.gov.ru:
(в репозитории находятся версии актуальные на 26 Июня 2015)
http://data.gov.ru/opendata/7710489036-dorozhkivelosipednye
http://data.gov.ru/opendata/7710489036-veloparkovki
http://data.gov.ru/opendata/7710489036-stantsiiprokatavelosipedov

Данные в немного разных нотациях, поэтому код по парсингу разный.

In [85]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.mpl_style', 'default')

import json


def parseRoads(filename='data/roads.csv'):
    """
        Generate list of [(Lat, Lon),...] roads
    """
    df = pd.read_csv(filename)
    roads = []
    for x in df['Track']:
        line = []
        for pair in x.split(';'):
            line.append(tuple([float(t) for t in pair.split(',')]))
        roads.append(line)
    return roads
            
def parsePark(filename='data/parks.csv'):
    """
        Generate list of (Lat, Lon) points with !nan coords
    """
    df = pd.read_csv(filename).dropna()
    def gen(dfs):
        return (t.replace(',', '.') for t in dfs)

    return zip(gen(df["X_WGS"]), gen(df["Y_WGS"]))

def parseRent(filename='data/rent.csv'):
    """
        Generate list of (Lat, Lon) points with !nan coords
    """
    df = pd.read_csv(filename).dropna()
    return zip(df["Longitude_WGS84"], df["Latitude_WGS84"])

# parseRoads()
# parsePark()
# parseRent()


Теперь у нас есть код, выдающий списки координат. Надо обратить внимание на разные нотации Lon-Lat/Lat-Lon/X-Y-Z(Mercator).
Далее у нас есть выбор, как эти точки рисовать. Быстрый путь -- преобразовать в JS код и нарисовать на какой-нибудь гуглокарте.
Для точек:
```
new google.maps.Marker({position: {lat, lon:},map: map});
```
Для путей:
_при желании, можно заинлайнить все жестко_
```
p = new google.maps.LatLng(Lat, Lon);
new google.maps.Polyline({path:p, geodesic: true, 
                          strokeColor: '#11FF00',\
                          strokeOpacity: 1.0, 
                          strokeWeight: 2, 
                          map: map});
```

### Рабочекрестьянский подход

На странице http://labs.strava.com/heatmap/#12/37.58191/55.74276/blue/bike в JS консоль вставляем код:
Вместо {{DATA}} подставить выхлоп json.dumps(roads)
```
var roads = {{DATA}};
for(var i in roads) {
    var path = [];
    for(var j in roads[i]) {
        path.push(new google.maps.LatLng(roads[i][j][1], roads[i][j][0]));
    }
    new google.maps.Polyline({path:path, geodesic: true, 
                              strokeColor: '#11FF00',
                              strokeOpacity: 1.0, 
                              strokeWeight: 2, 
                              map: map});
}
```
Для рисования точек такой метод: Вместо {{DATA}} подставить выхлоп json.dumps(points)
```
var points = {{DATA}};
for(var i in points) {
    var point = new google.maps.LatLng(points[i][1], points[i][0]);
    new google.maps.Marker({position:point,
                              map: map});
}
```

In [88]:

json.dumps(parseRent())


'[[37.600700000000003, 55.745399999999997], [37.6051, 55.7652], [37.622199999999999, 55.768000000000001], [37.639099999999999, 55.764699999999998], [37.647399999999998, 55.753300000000003], [37.635100000000001, 55.753900000000002], [37.630899999999997, 55.756799999999998], [37.636699999999998, 55.7654], [37.5899, 55.725700000000003], [37.584200000000003, 55.727400000000003], [37.579099999999997, 55.724200000000003], [37.586399999999998, 55.721699999999998], [37.575899999999997, 55.722000000000001], [37.583500000000001, 55.719200000000001], [37.578000000000003, 55.715499999999999], [37.561799999999998, 55.721800000000002], [37.569499999999998, 55.712600000000002], [37.560000000000002, 55.711599999999997], [37.555100000000003, 55.706200000000003], [37.545200000000001, 55.700400000000002], [37.541200000000003, 55.708799999999997], [37.535499999999999, 55.7149], [37.628599999999999, 55.741999999999997], [37.6252, 55.740699999999997], [37.617899999999999, 55.744799999999998], [37.5932999999